# Simple notebook for analyzing midway cis-eQTL effect on longevity

In [1]:
import pandas as pd

In [4]:
midway_survival_counts = pd.read_csv('../data/midway-survival-count.csv')
midway_model_table = pd.read_csv('../data/midway-survival-model-table.csv')


In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

mpl.rcParams["figure.facecolor"] = "w"
mpl.rcParams["figure.dpi"] = 150
mpl.rcParams["savefig.dpi"] = 600
mpl.rcParams["savefig.transparent"] = True
mpl.rcParams["font.size"] = 15
mpl.rcParams["font.family"] = "sans-serif"
mpl.rcParams["font.sans-serif"] = ["Arial", "DejaVu Sans"]
mpl.rcParams["axes.titlesize"] = "xx-large"  # medium, large, x-large, xx-large

mpl.style.use("seaborn-deep")

/var/folders/zp/2rs242nd655dzbrc8wb1jx300000gn/T/ipykernel_93921/1290679250.py:14: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  mpl.style.use("seaborn-deep")


In [37]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.duration.hazard_regression import PHReg

In [45]:
results = PHReg.from_formula('lifespan ~ C(sex) + C(genotype)', data=midway_model_table).fit()
results.summary()